# Dynamic programming 101

“Dynamic programming and structural estimation” mini course

Fedor Iskhakov


In [ ]:
pip install RISE # to make slideshow with Jupyter Notebooks

### What is Dynamic Programming

**“DP is recursive method for solving sequential decision problems”**

📖 Rust 2006, *New Palgrave Dictionary of Economics*

In computer science the meaning of the term is broader:
**DP is a general algorithm design technique for solving problems with
overlapping sub-problems.**

Generally allows solving in polynomial time ($ O(n^k) $) problems that would
otherwise take exponential time  ($ O(a^n) $)

## Example problem

**Tiling with dominoes**

Given a $ 3 \times n $ board, find **the number of ways** to
fill it with $ 2 \times 1 $ dominoes.

## Examples of tiling

These are three possible ways to fill up $ 3 \times 2 $ board

<img src="_static/tile1.jpg" style="height:100px;">

This is one possible way to tile $ 3 \times 8 $ board

<img src="_static/tile2.jpg" style="height:100px;">

The problem is to compute the number of possible tilings for any $ 3 \times n $ board.

## Breaking the big problem into subproblems

Observe that at any possible stage of filling up a $ 3 \times n $ board we may have
the following configurations

<img src="_static/tile3.jpg" style="height:180px;">

And it is impossible to have the following configurations

<img src="_static/tile4.jpg" style="height:150px;">

## Defining the recursion

The case of $ A_n $:

<img src="_static/tile5.jpg" style="height:120px;">

The case of $ B_n $:

<img src="_static/tile6.jpg" style="height:120px;">

The case of $ C_n $ is identical to $ B_n $, i.e. $ C_n = B_n $

## Recursive solution

Therefore for any $ n $ we have

$$
\begin{eqnarray*}
A_n &=& A_{n-2} + 2 B_{n-1} \\
B_n &=& A_{n-1} + B_{n-2}
\end{eqnarray*}
$$

The answer to the whole problem is given by $ A_n $

1. Inductive computation of the two sequences.  
1. Can be improved by *memoization* (optimization technique based on caching previous calls to the function)  

In [ ]:
def WaysTileDominoes(n):
    '''Compute the number of ways to tile 3 x n area by 2x1 tiles'''
    A = [0] * (n + 1)
    B = [0] * (n + 1)
    A[0] = 1 # one way to tile 3x0
    A[1] = 0 # no way to tile 3x1
    B[0] = 0 # no way to tile 3x0 without a corner
    B[1] = 1 # one way to tile 3x1 without a corner
    for i in range(2, n+1): # loop over 2,3,..,n
        A[i] = A[i-2] + 2 * B[i-1]
        B[i] = A[i-1] + B[i-2]
    return A[n]

In [ ]:
for n in range(1,20):
  print('There are',WaysTileDominoes(n),'to tile the 3 by',n,'board')

## Breaking the problem into sequence of small problems

- Thus, the sequential decision problem is broken into *initial decision*
  problem and the *future decisions* problem  
- The solution can be computed through **backward induction**,
  i.e. solving a sequential decision problem from the later periods  
- Embodiment of the recursive way of modeling sequential decisions is
  **Bellman equation**  

## Dynamic programming in economics

Many important problems and economic models are analyzed and solved
using dynamic programming:

- Dynamic models of labor supply  
- Job search  
- Human capital accumulation  
- Health process, insurance and long term care  
- Consumption/savings choices  
- Durable consumption  
- Growth models  
- Heterogeneous agents models  
- Overlapping generation models  

### Infinite horizon

- Tiling problem is inherently finite (Right?)
- For infinite problems things are a little different  
- How can we be sure that the algorithm would terminate?  

### Value of annuity

$$
\stackrel{\nearrow}{V} \quad
\stackrel{\searrow}{c} \quad
\stackrel{\searrow}{c} \quad
\stackrel{\searrow}{c} \quad
\dots
$$

- interest rate $ r $  
- What is the value of the annuity $ V $?  

### Value of annuity

$$
\stackrel{\nearrow}{V} \quad
\stackrel{\searrow}{c} \quad
\stackrel{\searrow}{c} \quad
\stackrel{\searrow}{c} \quad
\dots
$$

$$
V=\quad
\frac{c}{(1+r)^0} + \quad
\frac{c}{(1+r)^1} + \quad
\frac{c}{(1+r)^2} + \quad
\frac{c}{(1+r)^3} + \quad
\dots
$$

$$
\beta = \frac{1}{1+r}
$$

$$
V=\quad
c + \quad
c \beta + \quad
c \beta^2 + \quad
c \beta^3 + \quad
\dots
=
\sum_{t=0}^{\infty} \beta^t c
$$

### Answer by the geometric series formula

Assuming $ \beta<1 $

$$
V = \sum_{t=0}^{\infty} \beta^t c = \frac{c}{1-\beta}
$$

Can reformulate as **recursive equation**

$$
V = c + \beta ( c + \beta c + \beta^2 c + \dots ) = c + \beta V
$$

### Backward induction

1. Start with a guess $ V_0 $  
1. Insert into the Bellman equation  


$$
V_{i+1} = c + \beta V_i
$$

1. Repeat until convergence  


$$
||V_{i}-V_{i-1}||\leq\varepsilon\text{ (small number)}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

class annuity():
    def __init__(self,c=1,beta=.9):
        self.c = c           # Annual payment
        self.beta = beta     # Discount factor
        self.analytic = c/(1-beta)  # compute analytic solution right away
    def bellman(self,V):
        '''Bellman equation'''
        return self.c + self.beta*V
    def solve(self, maxiter = 1000, tol=1e-4, verbose=False):
        '''Solves the model using successive approximations'''
        if verbose: print('{:<4} {:>15} {:>15}'.format('Iter','Value','Error'))
        V0=0
        for i in range(maxiter):
            V1=self.bellman(V0)
            if verbose: print('{:<4d} {:>15.8f} {:>15.8f}'.format(i,V1,V1-self.analytic))
            if abs(V1-V0) < tol:
                break
            V0=V1
        else:  # when i went up to maxiter
            print('No convergence: maximum number of iterations achieved!')
        return V1

In [ ]:
a = annuity(c=10,beta=0.954)
print('Analytic solution is',a.analytic)
print('Numeric solution is ',a.solve())

### Questions to think about

- How many iterations are needed for computing the solution  
- How does this number depend on model parameters  